In [1]:
import os
import matplotlib.pyplot as plt
import copy
import time


import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import transforms, utils
from torchvision import datasets, models, transforms
torch.__version__

import pandas as pd
import numpy as np
from skimage import io, transform
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [3]:
!unzip '/content/drive/My Drive/Vision_training/color_train_33K.zip'
data = pd.read_csv('/content/drive/My Drive/Vision_training/color_train_33K.csv')
data = data[data.Color.notna()]
data[:30000].to_csv('./train_data.csv', index=False)
data[30000:32000].to_csv('./val_data.csv', index=False)
data[32000:].to_csv('./test_data.csv', index=False)

In [14]:
data

,ID,Color,img_index
0,45807645,Blue,1
1,45763067,Blue,10
2,45788939,Blue,4
3,45801298,White,6
4,45733396,Blue,4
...,...,...,...
33321,45762944,Green,5
33322,45784474,Blue,1
33323,45528775,Black,2
33324,45799215,White,6


In [24]:
colors = np.array(data.Color.unique())

class MyautoDataset_colors(Dataset):
  def __init__(self, csv_file, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.df = pd.read_csv(csv_file)
        self.transform = transform
        self.colors = colors

  def __len__(self):
    return len(self.df)

  def __getitem__(self, idx):
    if torch.is_tensor(idx):
        idx = idx.tolist()
    path = '/content/color_train_33K/{}_{}.jpg'.format(self.df.iloc[idx].ID, self.df.iloc[idx].img_index)
    image = transform.resize(io.imread(path), (224, 224))/255
    # y = [0]*len(self.colors)
    # y[np.where(self.colors == self.df.iloc[idx].Color)[0][0]] = 1
    y = np.where(self.colors == self.df.iloc[idx].Color)[0]
    sample = (torch.Tensor(np.einsum('ijk->kij',image)), torch.Tensor(y).long())

    # if self.transform:
    #     sample = self.transform(sample)

    return sample

In [25]:
colors

array(['Blue', 'White', 'Red', 'Silver', 'Grey', 'Black', 'Green'],
      dtype=object)

In [26]:
train = MyautoDataset_colors('./train_data.csv', '/content/training_data')
val = MyautoDataset_colors('./val_data.csv', '/content/training_data')
test = MyautoDataset_colors('./test_data.csv', '/content/training_data')

In [27]:
train_loader = DataLoader(train, batch_size=4,shuffle=True, num_workers=4)
val_loader = DataLoader(val, batch_size=4,shuffle=True, num_workers=4)
test_loader = DataLoader(test, batch_size=4,shuffle=True, num_workers=4)

In [28]:
dataloaders = {'train': train_loader, 'val': val_loader, 'test': test_loader}
dataset_sizes = {'train': len(train_loader), 'val': len(val_loader), 'test': len(test_loader)}

In [66]:
class model_inc(nn.Module):
  def __init__(self):
    super(model_inc, self).__init__()
    self.layers = nn.ModuleList()
    self.layers.append(models.resnet18(pretrained=True))
    self.layers.append(nn.Linear(1000, 256)) 
    self.layers.append(nn.Dropout(0.1))
    self.layers.append(nn.Linear(256, 32))
    self.layers.append(nn.Sigmoid())
    self.layers.append(nn.Dropout(0.1))
    self.layers.append(nn.Linear(32, len(data.Color.unique())))
    self.layers.append(nn.Softmax())
    
  def forward(self, x):
    for layer in self.layers:
      x = layer(x)
    return x

In [67]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0
            counter = 0
            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                counter += 1
                if counter % 100 == 0:
                  print(counter)
                  if counter % 1000 == 0:
                    print(running_corrects.double()/counter/4)
                inputs = inputs.to(device)
                labels = labels.to(device)
                labels = labels.reshape((labels.shape[0]))
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    # torch.max(outputs, dim=1)
                    preds = torch.argmax(outputs, 1)
                    # _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels.reshape((-1,)))

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss/4, epoch_acc/4))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc/4))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [68]:
criterion = nn.CrossEntropyLoss()
model = model_inc()
model.to(device)
# Observe that all parameters are being optimized
optimizer = optim.SGD(model.parameters(), lr=0.002, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.8)


In [69]:
model = train_model(model, criterion, optimizer, exp_lr_scheduler, num_epochs=5)

Epoch 0/4
----------


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  app.launch_new_instance()


100
200
300
400
500
600
700
800
900
1000
tensor(0.2903, device='cuda:0', dtype=torch.float64)
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
tensor(0.3749, device='cuda:0', dtype=torch.float64)
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
tensor(0.4391, device='cuda:0', dtype=torch.float64)
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
tensor(0.4694, device='cuda:0', dtype=torch.float64)
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
tensor(0.4922, device='cuda:0', dtype=torch.float64)
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
tensor(0.5062, device='cuda:0', dtype=torch.float64)
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
tensor(0.5221, device='cuda:0', dtype=torch.float64)
7100
7200
7300
7400
7500
train Loss: 1.6532 Acc: 0.5280
100
200
300
400
500
val Loss: 1.5248 Acc: 0.6520

Epoch 1/4
----------
100
200
300
400
500
600
700
800
900
1000
tensor(0.6468, device='cuda:0', dtype=torch.float64)
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
tensor(0.6485, de

In [75]:
model = train_model(model, criterion, optimizer, exp_lr_scheduler, num_epochs=2)

Epoch 0/1
----------


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  app.launch_new_instance()


100
200
300
400
500
600
700
800
900
1000
tensor(0.7835, device='cuda:0', dtype=torch.float64)
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
tensor(0.7850, device='cuda:0', dtype=torch.float64)
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
tensor(0.7812, device='cuda:0', dtype=torch.float64)
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
tensor(0.7828, device='cuda:0', dtype=torch.float64)
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
tensor(0.7817, device='cuda:0', dtype=torch.float64)
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
tensor(0.7841, device='cuda:0', dtype=torch.float64)
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
tensor(0.7844, device='cuda:0', dtype=torch.float64)
7100
7200
7300
7400
7500
train Loss: 1.3841 Acc: 0.7831
100
200
300
400
500
val Loss: 1.3980 Acc: 0.7695

Epoch 1/1
----------
100
200
300
400
500
600
700
800
900
1000
tensor(0.7935, device='cuda:0', dtype=torch.float64)
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
tensor(0.7968, de

In [76]:
torch.save(model.state_dict(), '/content/drive/My Drive/Myauto_vision/model_color_77.pt')